# 5.2 신경망 깊게 쌓아 컬러 데이터셋에 적용하기
Convolutional Neural Network (CNN) 을 쌓아올려 딥한 러닝을 해봅시다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
torch.manual_seed(42)
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

## 데이터셋 불러오기

In [4]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))])
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=True,
                   download=True,
                   transform=transform),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=False, 
                   transform=transform),
    batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified


## 뉴럴넷으로 Fashion MNIST 학습하기

In [5]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64,  2, stride=1)
        self.layer2 = self._make_layer(block, 128, 2, stride=2)
        self.layer3 = self._make_layer(block, 256, 2, stride=2)
        self.layer4 = self._make_layer(block, 512, 2, stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock)


## 하이퍼파라미터 

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다. 일반적으로 CPU 1개만 사용할 경우 필요는 없지만, GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다. 지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [6]:
model     = ResNet18().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.01)

## 훈련하기

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

## 테스트하기

아무리 훈련이 잘 되었다고 해도 실제 데이터를 만났을때 성능이 낮다면 쓸모 없는 모델일 것입니다. 우리가 진정 원하는 것은 훈련 데이터에 최적화한 모델이 아니라 모든 데이터에서 높은 성능을 보이는 모델이기 때문입니다. 세상에 존재하는 모든 데이터에 최적화 하는 것을 "일반화"라고 부르고 모델이 얼마나 실제 데이터에 적응하는지를 수치로 나타낸 것을 "일반화 오류"(Generalization Error) 라고 합니다. 

우리가 만든 모델이 얼마나 일반화를 잘 하는지 알아보기 위해, 그리고 언제 훈련을 멈추어야 할지 알기 위해 매 이포크가 끝날때 마다 테스트셋으로 모델의 성능을 측정해보겠습니다.

In [8]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # sum up batch loss
            test_loss += F.cross_entropy(output, target,
                                         size_average=False).item()

            # get the index of the max log-probability
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 훈련이 되는지 확인해봅시다!

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.346486
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.781493
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.462888
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.427824
[1] Test Loss: 1.2828, Accuracy: 54.95%
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.175778
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.431858
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.757462
Train Epoch: 2 [38400/50000 (77%)]	Loss: 0.960510
[2] Test Loss: 1.0339, Accuracy: 64.79%
Train Epoch: 3 [0/50000 (0%)]	Loss: 0.896164
Train Epoch: 3 [12800/50000 (26%)]	Loss: 0.528886
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.431045
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.592155
[3] Test Loss: 0.8495, Accuracy: 71.06%
Train Epoch: 4 [0/50000 (0%)]	Loss: 0.654060
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.657335
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.601032
Train Epoch: 4 [38400/50000 (77%)]	Loss: 0.566090
[4] Test Loss: 0.7315, Accuracy: 75.64%
Train Epoch: 5 [0/50000 (0%)]	Loss: 0.263753
Train Epoch: 5 

Train Epoch: 35 [25600/50000 (51%)]	Loss: 0.043934
Train Epoch: 35 [38400/50000 (77%)]	Loss: 0.014713
[35] Test Loss: 1.1658, Accuracy: 81.52%
Train Epoch: 36 [0/50000 (0%)]	Loss: 0.017344
Train Epoch: 36 [12800/50000 (26%)]	Loss: 0.000506
Train Epoch: 36 [25600/50000 (51%)]	Loss: 0.002387
Train Epoch: 36 [38400/50000 (77%)]	Loss: 0.003844
[36] Test Loss: 1.1801, Accuracy: 82.19%
Train Epoch: 37 [0/50000 (0%)]	Loss: 0.001715
Train Epoch: 37 [12800/50000 (26%)]	Loss: 0.011111
Train Epoch: 37 [25600/50000 (51%)]	Loss: 0.004604
Train Epoch: 37 [38400/50000 (77%)]	Loss: 0.065278
[37] Test Loss: 1.1691, Accuracy: 82.57%
Train Epoch: 38 [0/50000 (0%)]	Loss: 0.012695
Train Epoch: 38 [12800/50000 (26%)]	Loss: 0.002853
Train Epoch: 38 [25600/50000 (51%)]	Loss: 0.056870
Train Epoch: 38 [38400/50000 (77%)]	Loss: 0.000179
[38] Test Loss: 1.2270, Accuracy: 82.09%
Train Epoch: 39 [0/50000 (0%)]	Loss: 0.029769
Train Epoch: 39 [12800/50000 (26%)]	Loss: 0.113176
Train Epoch: 39 [25600/50000 (51%)]	Loss